# 05 Risk Scoring





In [ ]:
import pandas as pd
import numpy as np


In [ ]:
df = pd.read_csv("../data/processed/text_with_narratives.csv")
df.head()


In [ ]:
assert "topic" in df.columns
assert "clean_text" in df.columns


In [ ]:
narrative_size = df["topic"].value_counts().to_dict()
df["narrative_size"] = df["topic"].map(narrative_size)
df.head()


In [ ]:
np.random.seed(42)
df["misinformation_prob"] = np.random.uniform(0.4, 1.0, size=len(df))


In [ ]:
df["norm_narrative_size"] = (
    df["narrative_size"] - df["narrative_size"].min()
) / (
    df["narrative_size"].max() - df["narrative_size"].min()
)


In [ ]:
Risk Score =
  (0.6 × misinformation probability)
+ (0.4 × narrative spread)


In [ ]:
df["risk_score"] = (
    0.6 * df["misinformation_prob"]
    + 0.4 * df["norm_narrative_size"]
)


In [ ]:
def risk_level(score):
    if score >= 0.75:
        return "High Risk"
    elif score >= 0.5:
        return "Medium Risk"
    else:
        return "Low Risk"

df["risk_level"] = df["risk_score"].apply(risk_level)


In [ ]:
df[df["risk_level"] == "High Risk"][[
    "clean_text", "topic", "risk_score"
]].head()


In [ ]:
narrative_risk = (
    df.groupby("topic")
      .agg(
          avg_risk_score=("risk_score", "mean"),
          narrative_size=("clean_text", "count")
      )
      .reset_index()
      .sort_values(by="avg_risk_score", ascending=False)
)

narrative_risk.head()


In [ ]:
df.to_csv("../data/processed/text_with_risk_scores.csv", index=False)
narrative_risk.to_csv("../data/processed/narrative_risk_summary.csv", index=False)

print("Risk scoring results saved successfully.")
